In [2]:
%cd "~/projects/spartDM"
%load_ext autoreload
%autoreload 2

import os

print(f"Current working dir: {os.getcwd()}")

/home/kdang/projects/spartDM
Current working dir: /home/kdang/projects/spartDM


/home/kdang/projects/spartDM/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/kdang/projects/spartDM/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# ViVit models

In [27]:
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
import torchvision
from einops import rearrange, repeat
from matplotlib.gridspec import GridSpec
from sklearn import svm

from monai.data import DataLoader
from src.data.datamodules.starmen import StarmenDataset
from src.ldae.modules.ocsvm import standardize
from src.ldae.utils import feature_distance, heat_map, patchify
from src.utils.metrics import mae, mse, percnorm
from src.utils.visualization import (
    draw_featmap,
    filter_gt_ano_region,
    plot_comparison_starmen,
    plot_kde_pixel,
)
from src.ldae.nets import ViViT

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Import Data

from monai.utils.misc import first

train_ds = StarmenDataset(
    data_dir="data/starmen/output_random_noacc",
    split="train",
    nb_subject=None,
)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=1)

test = first(train_loader)
test.keys()
x_prev = test["x_prev"]
print(f"x_prev shape: {x_prev.shape}")
x = test["x"]
print(f"x shape: {x.shape}")

x_prev shape: torch.Size([2, 9, 1, 64, 64])
x shape: torch.Size([2, 1, 64, 64])


In [66]:
# Init ViVit model
vivit = ViViT((64, 64), (16, 16), 9, depth=1)

# Forward pass
out = vivit(x_prev)
out.shape

# Visualize model forward pass
from torch.utils.tensorboard import SummaryWriter

# Create SummaryWriter
writer = SummaryWriter(log_dir="workdir/debug/vivit")

# Add model graph to TensorBoard
writer.add_graph(vivit, x_prev)

writer.close()

